In [ ]:
!pip install -q -U ratchada-utils

In [ ]:
!pip show ratchada_utils

In [ ]:
from ratchada_utils.evaluator import simple_evaluator

result = pd.read_csv("./output/result-whisper-ratchada.csv")

summary = simple_evaluator(result["pred_text"], result["true_text"])

In [ ]:
summary